In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import os
import json
import datetime
import dateutil.parser

client = MongoClient('localhost', 27017)

db=client["DM_project_database"]

mycol=db["A12 (Genova-Roma)"]

week_dict={
        "lunedì":["2019-05-06","2019-05-13","2019-05-20","2019-05-27"],
        "martedì":["2019-05-07","2019-05-14","2019-05-21","2019-05-28"],
        "mercoledì":["2019-05-01","2019-05-08","2019-05-15","2019-05-22","2019-05-29"],
        "giovedì":["2019-05-02","2019-05-09","2019-05-16","2019-05-23","2019-05-30"],
        "venerdì":["2019-05-03","2019-05-10","2019-05-17","2019-05-24","2019-05-31"],
        "sabato":["2019-05-04","2019-05-11","2019-05-18","2019-05-25"],
        "domenica":["2019-05-05","2019-05-12","2019-05-19","2019-05-26"]
    }

def get_coordinates(file_name):
    mongo_coord=[]
    directory="C:/Users/dasan/Desktop"
    with open(directory+"/"+file_name) as file:
            data=json.load(file)
            
            for i in range(len(data['places'])):
                mongo_coord.append({
                                      "highway":"{}".format(data['places'][i]['highway']),
                                      "latitude":"{}".format(data['places'][i]['latitude']),
                                      "longitude":"{}".format(data['places'][i]['longitude'])})
    file.close()
    return mongo_coord


def find_place(place_name,highway_name):
    with open("C:/Users/dasan/Desktop/coordinates_file.json") as file:
        data=json.load(file)
        #print(data['places'])
        for i in range(len(data['places'])):
            if(data['places'][i]['city']==place_name and data['places'][i]['highway']== highway_name):
                return data['places'][i]['latitude'],data['places'][i]['longitude']


def find_minimum_base_time(subfolder):
    
    min_list=[]
    
    directory="C:/Users/dasan/Desktop/DM_project_final_Lagano/{}".format(subfolder)
    for filename in sorted(os.listdir(directory)):
        if(filename.endswith(".csv")):
            df=pd.read_csv(directory+"/"+filename)
            min_base_time_in_file=min(df["BASE TIME (minutes)"])
            min_list.append(min_base_time_in_file)
            
    return min(min_list)


def upload_data(subfolder):
    
    my_key=""
    
    directory="C:/Users/dasan/Desktop/DM_project_final_Lagano/{}".format(subfolder)

    mongo_list=[]

    for filename in sorted(os.listdir(directory)):
        if(filename.endswith(".csv")):
            df=pd.read_csv(directory+"/"+filename)
            for i in range(len(df["TIMESTAMP"])):
                string= df.iloc[i]["TIMESTAMP"]
                split=string.split(" ")
                date=split[0]
                time=split[1]
                
                for key in week_dict.keys():
                    for value in week_dict[key]:
                        if(value==date):
                            my_key=key
                            
                mongo_list.append({
                    "date":datetime.datetime.strptime(date,"%Y-%m-%d"),
                    "time":datetime.datetime.strptime(time,"%H:%M:%S"),
                    "day_of_week":my_key,
                    "distance_km":df.iloc[i]["DISTANCE (Km)"],
                    "delay_minutes":df.iloc[i]["TRAFFIC TIME (minutes)"]-find_minimum_base_time(subfolder)})
        my_key=""
    return mongo_list
    
if __name__=="__main__":
    mongo_tot=[]
    places={ "Rosignano - Pisa Centro":["Rosignano","Pisa Centro"]}
    
    mongo_1=upload_data("A12_Rosignano_PisaCentro")
    #mongo_2=upload_data("A4_SanGiorgio_Trieste")
    
    mongo_tot.append(mongo_1)
    #mongo_tot.append(mongo_2)
    
    print("Done")
    
    keys_list=list(places.keys())
    
        
    for i in range(len(mongo_tot)):
        #if(i==0):
         #   mycol.insert_one({
          #      "_id": "A32_coordinates",
           #     "coordinates": get_coordinates("A32.json")
           # })
        mycol.insert_one({
            "_id":keys_list[i],
            "origin_latitude":float(find_place(places.get(keys_list[i])[0],"A12(Genova-Roma)")[0]),
            "origin_longitude":float(find_place(places.get(keys_list[i])[0],"A12(Genova-Roma)")[1]),
            "destination_latitude":float(find_place(places.get(keys_list[i])[1],"A12(Genova-Roma)")[0]),
            "destination_longitude":float(find_place(places.get(keys_list[i])[1],"A12(Genova-Roma)")[1]),
            "traffic_info":mongo_tot[i]         
            })

Done
